Importação de Bibliotecas e Módulos

In [ ]:
import pandas as pd #Manipulação de conjuntos de dados
import numpy as np #Manipulações numéricas
import matplotlib.pyplot as plt #Análises gráficas
import seaborn as sns #Análises gráficas
import io #Manipulação de funções do sistema
import os #Manipulação de funções do sistema
import sys #Manipulação de funções do sistema

sys.path.insert(0, os.getcwd()) #Inicialização da pasta do projeto

from recursos import treino, teste #Importação dos Conjuntos
from matplotlib.widgets import TextBox #Visualização de textos

Verificação dos Conjuntos de Dados

In [ ]:
dfTreino = treino()
dfTreino.head()

In [ ]:
dfTest = teste()
dfTest.head()

Visualização das Informações dos Conjuntos

In [ ]:
def buildInfo(df):
  registros, colunas = df.shape

  info1 = f"""
  REGISTROS: {registros}
  COLUNAS: {colunas}
  """

  buffer = io.StringIO()
  df.info(buf=buffer)
  info2 = buffer.getvalue()

  return info1, info2

fig, ax = plt.subplots(nrows= 1,
                       ncols= 2,
                       figsize= (14, 8),
                       gridspec_kw= {'wspace': 0.4, 'hspace': 0.4})

ax1, ax2 = ax

label, initial = buildInfo(dfTreino)
textBox = TextBox(ax1, label= label, initial= initial)
ax1.set_title('CONJUNTO DE TREINO')

label, initial = buildInfo(dfTest)
textBox = TextBox(ax2, label= label, initial= initial)
ax2.set_title('CONJUNTO DE TESTE')

plt.show()

Alteração de Tipos de Variáveis

In [ ]:
dfTreino["ano_modelo"] = dfTreino["ano_modelo"].astype(int)
dfTest["ano_modelo"] = dfTest["ano_modelo"].astype(int)

Verificação de Dados Nulos ou Duplicados

In [ ]:
def isNullOrDuplicates(df):
  tamanho, _ = df.shape

  nulos = df.isnull().sum()
  nulosPct = (nulos / tamanho) * 100

  duplicados = df[df.duplicated()].sum()
  duplicadosPct = (duplicados / tamanho) * 100

  DATA = [nulos, nulosPct, duplicados, duplicadosPct]
  MAP = {0:"Nulos",1:"Nulos %",2:"Duplicados",3:"Duplicados %"}

  return pd.DataFrame(data=DATA).T.rename(columns=MAP)

In [ ]:
display(isNullOrDuplicates(dfTreino))

In [ ]:
display(isNullOrDuplicates(dfTest))

Preenchimento de Valores Nulos

In [ ]:
def fillFeatures(df, feature):
  criterio = df[feature].mode() if feature == "moda" else 0
  return df[feature].fillna(criterio)

In [ ]:
COLS_NULAS = ["num_fotos",
              "dono_aceita_troca",
              "veiculo_único_dono",
              "revisoes_concessionaria",
              "ipva_pago",
              "veiculo_licenciado",
              "garantia_de_fábrica"]

fillFeatures(dfTreino, *COLS_NULAS)
fillFeatures(dfTest, *COLS_NULAS)

 Remoção de Features Não Utilizadas

In [ ]:
dfTreino = dfTreino.drop(columns=['id','veiculo_alienado','elegivel_revisao'])
dfTest = dfTest.drop(columns=['id','veiculo_alienado','elegivel_revisao'])

Codificação Ordinal das Variáveis Categóricas

In [ ]:
def buildCodes(colecao):
  dictMap = {}
  for codigo, valor in enumerate(colecao):
    dictMap[valor] = codigo
  return dictMap

def ordinalCodes(df,colunas,valoresCategorias):
  for coluna, categoria in zip(colunas,valoresCategorias):
    mapeamento = buildCodes(categoria)
    df[coluna] = df[coluna].map(mapeamento)
  return df

def viewValues(df,colecao):
  listaSaida = []
  for coluna in colecao:
    valores = df[coluna].unique()
    listaSaida.append(valores)
  return listaSaida

dfTreino = ordinalCodes(dfTreino,COLUNAS_CAT,viewValues(dfTreino,COLUNAS_CAT))
dfTest = ordinalCodes(dfTest,COLUNAS_CAT,viewValues(dfTest,COLUNAS_CAT))

Refatoramento da Feature versao

Exportação dos Conjuntos Tratados

In [ ]:
PATH = "'\'recursos'\'" if os.name == "Windows" else "/recursos/"

treino = os.getcwd() + PATH + 'cars_train_clean.csv'
dfTreino.to_csv(treino,sep="\t",encoding='UTF-16')

teste = os.getcwd() + PATH + 'cars_test_clean.csv'
dfTest.to_csv(teste,sep="\t",encoding='UTF-16')